<a href="https://colab.research.google.com/github/LukmaanViscomi/AI-Deep-Learning/blob/main/_Baseline_Controlled_V3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### PHASE-1 - UNZIP THE DATA

In [1]:
!apt-get install p7zip-full # Install 7-Zip
!pip install patool # Install the patool library which provides the patoolib module
import zipfile
import os
import patoolib # Now you can import patoolib

# Path to the uploaded zip file
zip_file_path = 'dataset2 (1).zip'
extracted_folder_path = './dataset2'  # Use a relative path for the extraction directory

# Extract the zip file using patool
patoolib.extract_archive(zip_file_path, outdir=extracted_folder_path)

# List the contents of the extracted folder
extracted_files = os.listdir(extracted_folder_path)
print(extracted_files)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 6.9 MB/s eta 0:00:00


INFO patool: Extracting dataset2 (1).zip ...
INFO:patool:Extracting dataset2 (1).zip ...
INFO patool: ... creating output directory `./dataset2'.
INFO:patool:... creating output directory `./dataset2'.
INFO patool: running /usr/bin/7z x -o./dataset2 -- "dataset2 (1).zip"
INFO:patool:running /usr/bin/7z x -o./dataset2 -- "dataset2 (1).zip"
INFO patool:     with input=''
INFO:patool:    with input=''
INFO patool: ... dataset2 (1).zip extracted to `./dataset2'.
INFO:patool:... dataset2 (1).zip extracted to `./dataset2'.


['triple_mnist']


### PHASE-2 - REDISTRIBUTE THE DATASET FOR TRAIN-VAL-TEST

In [2]:
import os
import shutil
from pathlib import Path
import random

# Paths to original directories
original_base_dir = Path('dataset2/triple_mnist')
original_train_dir = original_base_dir / 'train'
original_val_dir = original_base_dir / 'val'
original_test_dir = original_base_dir / 'test'

# Path to the new dataset directory
new_base_dir = Path('dataset-c/triple_mnist')
new_train_dir = new_base_dir / 'train'
new_val_dir = new_base_dir / 'val'
new_test_dir = new_base_dir / 'test'

# Ensure the new directories exist
new_train_dir.mkdir(parents=True, exist_ok=True)
new_val_dir.mkdir(parents=True, exist_ok=True)
new_test_dir.mkdir(parents=True, exist_ok=True)

# Function to split and copy files
def split_and_copy_files(src_dir, new_train_dir, new_val_dir, new_test_dir, train_ratio=0.6, val_ratio=0.2, test_ratio=0.2):
    if not src_dir.exists():
        return

    classes = sorted(os.listdir(src_dir))

    for cls in classes:
        cls_path = src_dir / cls
        if cls_path.is_dir():
            images = list(cls_path.glob('*'))
            random.shuffle(images)

            num_train = int(len(images) * train_ratio)
            num_val = int(len(images) * val_ratio)

            train_images = images[:num_train]
            val_images = images[num_train:num_train+num_val]
            test_images = images[num_train+num_val:]

            cls_train_dir = new_train_dir / cls
            cls_val_dir = new_val_dir / cls
            cls_test_dir = new_test_dir / cls

            cls_train_dir.mkdir(parents=True, exist_ok=True)
            cls_val_dir.mkdir(parents=True, exist_ok=True)
            cls_test_dir.mkdir(parents=True, exist_ok=True)

            for img in train_images:
                shutil.copy(str(img), str(cls_train_dir / img.name))
            for img in val_images:
                shutil.copy(str(img), str(cls_val_dir / img.name))
            for img in test_images:
                shutil.copy(str(img), str(cls_test_dir / img.name))

# Split and copy files from original train, val, and test directories
split_and_copy_files(original_train_dir, new_train_dir, new_val_dir, new_test_dir)
split_and_copy_files(original_val_dir, new_train_dir, new_val_dir, new_test_dir)
split_and_copy_files(original_test_dir, new_train_dir, new_val_dir, new_test_dir)

print("Files split and copied successfully!")

Files split and copied successfully!


### VERISON 3.0 - LETS GET CRACKING!!!

### Section 1: Imports and Initial Setup

In [3]:
!pip install keras-tuner

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt
import os
import shutil
import random
import matplotlib.pyplot as plt
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 9.4 MB/s eta 0:00:00


### Section 2: Function to Create a Small Subset of the Dataset

In [4]:
### Section 2. Function to Create a Small Subset of the Dataset
def create_subset(original_dir, subset_dir, classes, num_images_per_class):
    if not os.path.exists(subset_dir):
        os.makedirs(subset_dir)
    for class_name in classes:
        class_dir = os.path.join(original_dir, class_name)
        subset_class_dir = os.path.join(subset_dir, class_name)
        if not os.path.exists(subset_class_dir):
            os.makedirs(subset_class_dir)
        images = os.listdir(class_dir)
        random.shuffle(images)
        for img in images[:num_images_per_class]:
            shutil.copy(os.path.join(class_dir, img), os.path.join(subset_class_dir, img))



### Section 3: Paths to Your Full and Subset Datasets


In [10]:
### 2. Paths to your full dataset
full_train_dir = 'dataset-c/triple_mnist/train'
full_val_dir = 'dataset-c/triple_mnist/val'
full_test_dir = 'dataset-c/triple_mnist/test'

### Section 4. Paths to Your Subset Dataset
subset_train_dir = 'subset/triple_mnist/train'
subset_val_dir = 'subset/triple_mnist/val'
subset_test_dir = 'subset/triple_mnist/test'

### Section 4: Create a Small Subset of Your Data



In [11]:
### Create a small subset of your data
classes_to_use = [str(i).zfill(3) for i in range(50)]  # Use the first 50 classes as an example
num_images_per_class = 500  # Increase to 500 images per class
create_subset(full_train_dir, subset_train_dir, classes_to_use, num_images_per_class)
create_subset(full_val_dir, subset_val_dir, classes_to_use, num_images_per_class)
create_subset(full_test_dir, subset_test_dir, classes_to_use, num_images_per_class)


### Section 5. Image data generators for the subset (Updated for Data Augmentation)

---




In [12]:
### Section 5. Image data generators for the subset (Updated for Data Augmentation)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


old version

In [13]:
# ### 5. Image data generators for the subset
# train_datagen = ImageDataGenerator(rescale=1./255)
# val_datagen = ImageDataGenerator(rescale=1./255)
# test_datagen = ImageDataGenerator(rescale=1./255)

### Section 6: Data Generators for the Subset


In [14]:
### Section 6. Data Generators for the Subset
train_generator = train_datagen.flow_from_directory(
    subset_train_dir,
    target_size=(84, 84),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    subset_val_dir,
    target_size=(84, 84),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    subset_test_dir,
    target_size=(84, 84),
    batch_size=32,
    class_mode='categorical'
)

Found 3000 images belonging to 50 classes.
Found 1000 images belonging to 50 classes.
Found 1000 images belonging to 50 classes.


### Section 7: Transfer Learning (Optional)

In [15]:
### Section 7. Transfer Learning (Optional)
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(84, 84, 3))
for layer in base_model.layers:
    layer.trainable = False  # Freeze layers to use as feature extractor

model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


### Section 8: Hyperparameter Tuning Function with Batch Size, Batch Normalization, and Regularization

Version 2 is generally more robust due to the inclusion of batch normalization. It can lead to more stable training and better generalizatio

In [33]:
# from tensorflow.keras.layers import BatchNormalization
# from tensorflow.keras.regularizers import l2

# ### Section 8. Hyperparameter Tuning Function with Regularization and Batch Normalization
# def build_model(hp):
#     model = Sequential()
#     model.add(Conv2D(hp.Int('conv_1_filter', 32, 128, step=32), (3, 3), activation='relu',
#                      input_shape=(84, 84, 3), kernel_regularizer=l2(0.001)))
#     model.add(BatchNormalization())  # Added Batch Normalization
#     model.add(MaxPooling2D((2, 2)))
#     model.add(Conv2D(hp.Int('conv_2_filter', 32, 128, step=32), (3, 3), activation='relu', kernel_regularizer=l2(0.001)))
#     model.add(BatchNormalization())  # Added Batch Normalization
#     model.add(MaxPooling2D((2, 2)))
#     model.add(Flatten())
#     model.add(Dense(hp.Int('dense_units', 32, 128, step=32), activation='relu', kernel_regularizer=l2(0.001)))
#     model.add(Dropout(hp.Float('dropout_rate', 0.2, 0.5, step=0.1)))
#     model.add(Dense(train_generator.num_classes, activation='softmax'))

#     model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-2, sampling='LOG')),
#                   loss='categorical_crossentropy',
#                   metrics=['accuracy'])

#     return model


Version 1 might be preferred if you want to simplify the model and focus more on controlling overfitting through regularization alone.

In [31]:
# # Section 8. Hyperparameter Tuning Function with Batch Size, Batch Normalization, and Regularization
# def build_model(hp):
#     model = Sequential()

#     # Print the batch size being used for this trial
#     #print(f"Batch size for this trial: {hp.get('batch_size')}")

#     # First convolutional layer
#     model.add(Conv2D(hp.Int('conv_1_filter', 32, 128, step=32), (3, 3), activation='relu', input_shape=(84, 84, 3)))
#     model.add(MaxPooling2D((2, 2)))

#     # Second convolutional layer
#     model.add(Conv2D(hp.Int('conv_2_filter', 32, 128, step=32), (3, 3), activation='relu'))
#     model.add(MaxPooling2D((2, 2)))

#     model.add(Flatten())

#     # Dense layer with regularization
#     model.add(Dense(hp.Int('dense_units', 32, 128, step=32), activation='relu',
#                     kernel_regularizer=tf.keras.regularizers.l2(hp.Float('l2_regularization', 1e-4, 1e-2, sampling='LOG'))))

#     model.add(Dropout(hp.Float('dropout_rate', 0.2, 0.5, step=0.1)))

#     # Output layer
#     model.add(Dense(train_generator.num_classes, activation='softmax'))

#     # Compile the model with a tunable learning rate
#     model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-2, sampling='LOG')),
#                   loss='categorical_crossentropy',
#                   metrics=['accuracy'])

#     return model



new version to fix patch size issues

In [37]:
# Section 8. Hyperparameter Tuning Function with Batch Size, Batch Normalization, and Regularization
def build_model(hp):
    model = Sequential()

    # Print the batch size being used for this trial
    batch_size = hp.Int('batch_size', 16, 64, step=16)
    print(f"Batch size for this trial: {batch_size}")

    # First convolutional layer
    model.add(Conv2D(hp.Int('conv_1_filter', 32, 128, step=32), (3, 3), activation='relu', input_shape=(84, 84, 3)))
    model.add(MaxPooling2D((2, 2)))

    # Second convolutional layer
    model.add(Conv2D(hp.Int('conv_2_filter', 32, 128, step=32), (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())

    # Dense layer with regularization
    model.add(Dense(hp.Int('dense_units', 32, 128, step=32), activation='relu',
                    kernel_regularizer=tf.keras.regularizers.l2(hp.Float('l2_regularization', 1e-4, 1e-2, sampling='LOG'))))

    model.add(Dropout(hp.Float('dropout_rate', 0.2, 0.5, step=0.1)))

    # Output layer
    model.add(Dense(train_generator.num_classes, activation='softmax'))

    # Compile the model with a tunable learning rate
    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-2, sampling='LOG')),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


### Section 9: Early Stopping Callback

In [38]:
### Section 9. Callback for Early Stopping (Updated for longer patience)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,  # Increased patience for better fine-tuning
    restore_best_weights=True
)


### Section 10: Tuner Setup (Corrected with Batch Size Tuning)



Newest Version is probably the most straightforward while maintaining modularity. It ensures that batch size tuning is included without adding unnecessary complexity.

In [ ]:
# # Section 10. Tuner Setup (Including Batch Size Tuning)
# tuner = kt.Hyperband(
#     build_model,
#     objective='val_accuracy',
#     max_epochs=10,
#     factor=3,
#     directory='new_output',
#     project_name='digit_tuning_subset'
# )

# # Adding batch size to the hyperparameter search space
# hp = kt.HyperParameters()
# hp.Int('batch_size', 16, 64, step=16)

# # Include batch size in the search space and ensure it's passed correctly
# tuner.search(
#     train_generator,
#     epochs=10,
#     validation_data=val_generator,
#     callbacks=[early_stopping],
#     hyperparameters=hp
# )

# # Log the hyperparameters, including batch size
# tuner.search_space_summary()



Version 1 adds an extra step by retrieving the batch_size manually,

In [ ]:
# # Section 10. Tuner Setup (Including Batch Size Tuning)
# tuner = kt.Hyperband(
#     build_model,
#     objective='val_accuracy',
#     max_epochs=10,
#     factor=3,
#     directory='new_output',
#     project_name='digit_tuning_subset'
# )

# # Adding batch size to the hyperparameter search space
# hp = kt.HyperParameters()
# hp.Int('batch_size', 16, 64, step=16)

# # Log the hyperparameters, including batch size
# tuner.search_space_summary()

# # Start the hyperparameter search
# tuner.search(
#     train_generator,
#     epochs=10,
#     validation_data=val_generator,
#     callbacks=[early_stopping],
#     # Pass the batch size as part of the search space
#     batch_size=hp.get('batch_size')
# )


Version 2 is the most concise but lacks the flexibility and might be less clear when scaling or adding more hyperparameters.

In [ ]:
# # Section 10. Tuner Setup
# tuner = kt.Hyperband(
#     build_model,
#     objective='val_accuracy',
#     max_epochs=10,
#     factor=3,
#     directory='new_output',
#     project_name='digit_tuning_subset'
# )

# # Adding batch size to the hyperparameter search space
# tuner.search_space_summary()
# tuner.search(
#     train_generator,
#     epochs=10,
#     validation_data=val_generator,
#     callbacks=[early_stopping],
#     # Specify batch size here to be tuned
#     batch_size=kt.HyperParameters().Int('batch_size', 16, 64, step=16)
# )


new version

In [ ]:
# Section 10. Tuner Setup (Including Batch Size Tuning)
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='new_output',
    project_name='digit_tuning_subset'
)

# Start the hyperparameter search
tuner.search(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[early_stopping]
)

# Log the hyperparameters, including batch size
tuner.search_space_summary()


Trial 15 Complete [00h 00m 35s]
val_accuracy: 0.019999999552965164

Best val_accuracy So Far: 0.04100000113248825
Total elapsed time: 00h 07m 01s

Search: Running Trial #16

Value             |Best Value So Far |Hyperparameter
96                |32                |conv_1_filter
32                |128               |conv_2_filter
96                |96                |dense_units
0.3               |0.2               |dropout_rate
0.0035583         |0.00020196        |learning_rate
16                |48                |batch_size
0.0019894         |0.00015774        |l2_regularization
4                 |4                 |tuner/epochs
0                 |2                 |tuner/initial_epoch
1                 |2                 |tuner/bracket
0                 |1                 |tuner/round

Batch size for this trial: 16
Epoch 1/4
94/94 ━━━━━━━━━━━━━━━━━━━━ 13s 103ms/step - accuracy: 0.0192 - loss: 4.1663 - val_accuracy: 0.0200 - val_loss: 3.9208
Epoch 2/4
60/94 ━━━━━━━━━━━━━━━━━━━━ 2s 6

### Section 11: Learning Rate Scheduler (New)

In [ ]:
### Section 11. Learning Rate Scheduler
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=3,
    min_lr=1e-6,
    verbose=1
)


### Section 12: Perform the Hyperparameter Search on the Subset

In [ ]:
### Section 12. Perform the Hyperparameter Search on the Subset
tuner.search(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[early_stopping, lr_scheduler]  # Include both callbacks here
)


### Section 13: Train Model with Best Hyperparameters



In [ ]:
### Section 13. Train Model with Best Hyperparameters
print("Compiling and training the model with the best hyperparameters...")

# Build the best model from the hyperparameter search
best_model = tuner.hypermodel.build(best_hps)

# Explicitly compile the model to ensure metrics are tracked
best_model.compile(
    optimizer=Adam(learning_rate=best_hps.get('learning_rate')),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model with the best hyperparameters, using the early stopping and learning rate scheduler callbacks
history = best_model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[early_stopping, lr_scheduler]
)

# After training, print out the history keys to check for 'val_accuracy'
print("Available keys in history.history after training:", history.history.keys())

# Evaluate the model on the subset test data
test_loss, test_acc = best_model.evaluate(test_generator)
print(f"Test accuracy on subset: {test_acc}")


### Section 14: Cross-Validation Loop (Optional)


In [ ]:
### Section 14: Cross-Validation Loop (Optional)
from sklearn.model_selection import KFold

### Section 14. Cross-Validation Loop (Optional)
kf = KFold(n_splits=5)
cv_scores = []

for train_idx, val_idx in kf.split(train_data):
    train_data_fold = train_data[train_idx]
    val_data_fold = val_data[val_idx]

    history = best_model.fit(train_data_fold, epochs=10, validation_data=val_data_fold,
                             callbacks=[early_stopping, lr_scheduler])
    score = best_model.evaluate(val_data_fold)
    cv_scores.append(score)

print("Cross-validation scores:", cv_scores)


### Section 15: Get the Best Hyperparameters

In [ ]:
### Section 15. Get the Best Hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the winning hyperparameters on screen
print("Winning Hyperparameters:")
print(f"conv_1_filter: {best_hps.get('conv_1_filter')}")
print(f"conv_2_filter: {best_hps.get('conv_2_filter')}")
print(f"dense_units: {best_hps.get('dense_units')}")
print(f"dropout_rate: {best_hps.get('dropout_rate')}")
print(f"learning_rate: {best_hps.get('learning_rate')}")
print(f"batch_size: {best_hps.values.get('batch_size', 32)}")



### Section 16. Save the results in a CSV file

In [ ]:
### Section 16. Save the results in a CSV file
results = []
for trial in tuner.oracle.get_best_trials(num_trials=10):
    trial_summary = {
        'trial_number': trial.trial_id,
        'conv_1_filter': trial.hyperparameters.get('conv_1_filter'),
        'conv_2_filter': trial.hyperparameters.get('conv_2_filter'),
        'dense_units': trial.hyperparameters.get('dense_units'),
        'dropout_rate': trial.hyperparameters.get('dropout_rate'),
        'learning_rate': trial.hyperparameters.get('learning_rate'),
        'batch_size': trial.hyperparameters.values['batch_size'] if 'batch_size' in trial.hyperparameters.values else 32,
        'accuracy': trial.metrics.get_last_value('accuracy'),
        'loss': trial.metrics.get_last_value('loss'),
        'val_accuracy': trial.metrics.get_last_value('val_accuracy'),
        'val_loss': trial.metrics.get_last_value('val_loss'),
        'test_accuracy': test_acc,
        'test_loss': test_loss,
    }
    results.append(trial_summary)

results_df = pd.DataFrame(results)
results_df.to_csv('new_output/trial_results.csv', index=False)


### Section 17: Train on Full Dataset (Updated)

In [ ]:
### Section 17. Train on Full Dataset Without Validation Threshold

# Use the best hyperparameters for the full dataset
batch_size = best_hps.values.get('batch_size', 32)  # Properly retrieve the batch size with a default fallback

train_generator_full = train_datagen.flow_from_directory(
    full_train_dir,
    target_size=(84, 84),
    batch_size=batch_size,  # Use the resolved batch_size
    class_mode='categorical'
)

val_generator_full = val_datagen.flow_from_directory(
    full_val_dir,
    target_size=(84, 84),
    batch_size=batch_size,  # Use the resolved batch_size
    class_mode='categorical'
)

test_generator_full = test_datagen.flow_from_directory(
    full_test_dir,
    target_size=(84, 84),
    batch_size=batch_size,  # Use the resolved batch_size
    class_mode='categorical'
)

# Build and train the model with the best hyperparameters on the full dataset
best_model_full = tuner.hypermodel.build(best_hps)

# Modify the final layer to match the number of classes in the full dataset
best_model_full.pop()  # Remove the old final layer
best_model_full.add(Dense(train_generator_full.num_classes, activation='softmax'))  # Add a new layer with correct output size

best_model_full.compile(optimizer=Adam(learning_rate=best_hps.get('learning_rate')),
                        loss='categorical_crossentropy',
                        metrics=['accuracy'])

history_full = best_model_full.fit(
    train_generator_full,
    epochs=20,
    validation_data=val_generator_full,
    callbacks=[early_stopping]
)

# Evaluate the model on the full test data
test_loss_full, test_acc_full = best_model_full.evaluate(test_generator_full)
print(f"Test accuracy on full dataset: {test_acc_full}")

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_full.history['accuracy'], label='accuracy')
plt.plot(history_full.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_full.history['loss'], label='loss')
plt.plot(history_full.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Displaying the winning hyperparameters
best_hyperparameters_df = pd.DataFrame([{
    'conv_1_filter': best_hps.get('conv_1_filter'),
    'conv_2_filter': best_hps.get('conv_2_filter'),
    'dense_units': best_hps.get('dense_units'),
    'dropout_rate': best_hps.get('dropout_rate'),
    'learning_rate': best_hps.get('learning_rate'),
    'batch_size': batch_size  # Use the resolved batch size
}])
print(best_hyperparameters_df)
best_hyperparameters_df.to_csv('new_output/best_hyperparameters.csv', index=False)
